# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

In [3]:
# zをpreluに通す。


## load library

In [4]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [5]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/meta_dense/latent5_mixture3_revised_euclid_no4/config.yaml'

## make path list and train/valid split

In [8]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [9]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [10]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    #try:
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    # output
    model = history['model']
    gmm_param = history['gmm_param']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    gmm_param_path = MODEL_DIR+'/{}_gmm_param.pkl'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    pd.to_pickle(gmm_param, gmm_param_path)
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    #except:
    #    print("error")
    #    history = "error"
    return history

In [11]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [ ]:
for machine_type in machine_types:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-12-01 01:07:16,999 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-12-01 01:07:17,001 - 00_train.py - INFO - MAKE DATA_LOADER
2020-12-01 01:07:17,969 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 29/29 [00:02<00:00, 10.44it/s]
2020-12-01 01:07:48,312 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:95154.034241, tr_rec:94938.849182, tr_eng:2016.023170, tr_covd:2716.493735, val_AUC:0.536007, val_pAUC:0.507362
100%|██████████| 29/29 [00:02<00:00, 10.21it/s]
2020-12-01 01:08:14,882 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:89459.806030, tr_rec:89241.712646, tr_eng:2122.426733, tr_covd:1170.167935, val_AUC:0.512958, val_pAUC:0.511657
100%|██████████| 29/29 [00:03<00:00,  9.58it/s]
2020-12-01 01:08:42,076 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:85141.633118, tr_rec:84924.509949, tr_eng:2136.194382, tr_covd:700.811949, val_AUC:0.507140, val_pAUC:0.509603
100%|██████████| 29/29 [00:03<00:00,  9.47it/s]
2020-12-01 01:09:09,251 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:80037.642700, tr_rec:79822.166992, tr_eng:2128.456553, tr_covd:525.972281, val_AUC:0.529329, val_pAUC:0.504019
100%|██████████| 29/29 [00:02<00:00,  9.98it/s]
2020-1

100%|██████████| 29/29 [00:02<00:00, 10.11it/s]
2020-12-01 01:20:48,509 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:816.575275, tr_rec:654.538908, tr_eng:1611.471692, tr_covd:177.839196, val_AUC:0.533358, val_pAUC:0.504231
100%|██████████| 29/29 [00:02<00:00, 10.19it/s]
2020-12-01 01:21:15,305 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:1238.290625, tr_rec:645.896846, tr_eng:1624.285870, tr_covd:85993.042315, val_AUC:0.508876, val_pAUC:0.492591
100%|██████████| 29/29 [00:02<00:00, 10.34it/s]
2020-12-01 01:21:42,336 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:804.650511, tr_rec:633.519149, tr_eng:1704.186378, tr_covd:142.544487, val_AUC:0.483890, val_pAUC:0.489839
100%|██████████| 29/29 [00:02<00:00, 10.64it/s]
2020-12-01 01:22:09,238 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:948.531953, tr_rec:640.950517, tr_eng:1625.704712, tr_covd:29002.191520, val_AUC:0.478005, val_pAUC:0.500445
100%|██████████| 29/29 [00:02<00:00, 10.39it/s]
2020-12-01 01:2

100%|██████████| 29/29 [00:05<00:00,  5.62it/s]
2020-12-01 01:38:51,052 - pytorch_modeler.py - INFO - Epoch 59/300:train_loss:717.170405, tr_rec:570.681134, tr_eng:1453.234048, tr_covd:233.172685, val_AUC:0.536888, val_pAUC:0.504075
100%|██████████| 29/29 [00:04<00:00,  6.38it/s]
2020-12-01 01:39:40,425 - pytorch_modeler.py - INFO - Epoch 60/300:train_loss:11945.182909, tr_rec:576.259645, tr_eng:1515.995528, tr_covd:2243464.729485, val_AUC:0.509550, val_pAUC:0.508262
100%|██████████| 29/29 [00:05<00:00,  5.35it/s]
2020-12-01 01:40:29,961 - pytorch_modeler.py - INFO - Epoch 61/300:train_loss:621.172692, tr_rec:596.853525, tr_eng:232.656297, tr_covd:210.707527, val_AUC:0.468056, val_pAUC:0.504131
100%|██████████| 29/29 [00:05<00:00,  5.28it/s]
2020-12-01 01:41:19,616 - pytorch_modeler.py - INFO - Epoch 62/300:train_loss:720.539994, tr_rec:580.727579, tr_eng:1384.447179, tr_covd:273.537993, val_AUC:0.496144, val_pAUC:0.499694
100%|██████████| 29/29 [00:05<00:00,  5.37it/s]
2020-12-01 01:4

...

100%|██████████| 20/20 [00:03<00:00,  5.79it/s]
2020-12-01 09:23:56,438 - pytorch_modeler.py - INFO - Epoch 118/300:train_loss:2005521.767992, tr_rec:476.187482, tr_eng:-7670.178701, tr_covd:401162517.612823, val_AUC:0.676551, val_pAUC:0.652731
100%|██████████| 20/20 [00:03<00:00,  5.50it/s]
2020-12-01 09:24:33,589 - pytorch_modeler.py - INFO - Epoch 119/300:train_loss:577.547522, tr_rec:504.226024, tr_eng:-757.368057, tr_covd:29811.661388, val_AUC:0.502847, val_pAUC:0.569631
100%|██████████| 20/20 [00:03<00:00,  5.75it/s]
2020-12-01 09:25:10,381 - pytorch_modeler.py - INFO - Epoch 120/300:train_loss:593.968997, tr_rec:501.598028, tr_eng:16.829070, tr_covd:18137.612138, val_AUC:0.637423, val_pAUC:0.606725
100%|██████████| 20/20 [00:03<00:00,  5.66it/s]
2020-12-01 09:25:48,436 - pytorch_modeler.py - INFO - Epoch 121/300:train_loss:1142.848642, tr_rec:509.917502, tr_eng:209.749904, tr_covd:122391.232937, val_AUC:0.686718, val_pAUC:0.501548
100%|██████████| 20/20 [00:03<00:00,  5.75it/s]


100%|██████████| 20/20 [00:03<00:00,  6.62it/s]
2020-12-01 09:41:53,760 - pytorch_modeler.py - INFO - Epoch 147/300:train_loss:525.831830, tr_rec:435.799176, tr_eng:889.015525, tr_covd:226.220458, val_AUC:0.731065, val_pAUC:0.511237
100%|██████████| 20/20 [00:03<00:00,  5.69it/s]
2020-12-01 09:42:31,428 - pytorch_modeler.py - INFO - Epoch 148/300:train_loss:491.433857, tr_rec:438.250601, tr_eng:478.458167, tr_covd:1067.487293, val_AUC:0.742881, val_pAUC:0.520266
100%|██████████| 20/20 [00:03<00:00,  5.66it/s]
2020-12-01 09:43:08,155 - pytorch_modeler.py - INFO - Epoch 149/300:train_loss:482.945000, tr_rec:439.287996, tr_eng:426.030812, tr_covd:210.784543, val_AUC:0.716526, val_pAUC:0.548378
100%|██████████| 20/20 [00:03<00:00,  5.82it/s]
2020-12-01 09:43:45,193 - pytorch_modeler.py - INFO - Epoch 150/300:train_loss:463.342927, tr_rec:435.814445, tr_eng:257.274781, tr_covd:360.200540, val_AUC:0.714960, val_pAUC:0.497355
100%|██████████| 20/20 [00:03<00:00,  5.94it/s]
2020-12-01 09:44:22

100%|██████████| 20/20 [00:03<00:00,  5.69it/s]
2020-12-01 09:59:48,418 - pytorch_modeler.py - INFO - Epoch 176/300:train_loss:440.833249, tr_rec:421.298877, tr_eng:181.784215, tr_covd:271.190609, val_AUC:0.706944, val_pAUC:0.651565
100%|██████████| 20/20 [00:03<00:00,  6.19it/s]
2020-12-01 10:00:24,848 - pytorch_modeler.py - INFO - Epoch 177/300:train_loss:374.116194, tr_rec:418.000772, tr_eng:-476.472415, tr_covd:752.532756, val_AUC:0.689069, val_pAUC:0.601289
100%|██████████| 20/20 [00:03<00:00,  5.67it/s]
2020-12-01 10:01:03,164 - pytorch_modeler.py - INFO - Epoch 178/300:train_loss:364.642583, tr_rec:419.011593, tr_eng:-551.746967, tr_covd:161.137269, val_AUC:0.616906, val_pAUC:0.568045
100%|██████████| 20/20 [00:03<00:00,  5.69it/s]
2020-12-01 10:01:39,827 - pytorch_modeler.py - INFO - Epoch 179/300:train_loss:-300.122897, tr_rec:418.416058, tr_eng:-7192.955929, tr_covd:151.330877, val_AUC:0.717125, val_pAUC:0.665687
100%|██████████| 20/20 [00:03<00:00,  5.73it/s]
2020-12-01 10:0

100%|██████████| 20/20 [00:03<00:00,  5.55it/s]
2020-12-01 10:18:05,120 - pytorch_modeler.py - INFO - Epoch 205/300:train_loss:-190.019996, tr_rec:403.838800, tr_eng:-5942.772941, tr_covd:83.702551, val_AUC:0.791640, val_pAUC:0.543486
100%|██████████| 20/20 [00:03<00:00,  5.63it/s]
2020-12-01 10:18:42,384 - pytorch_modeler.py - INFO - Epoch 206/300:train_loss:381.792301, tr_rec:400.005604, tr_eng:-187.770191, tr_covd:112.742717, val_AUC:0.716436, val_pAUC:0.630755
100%|██████████| 20/20 [00:03<00:00,  5.54it/s]
2020-12-01 10:19:18,771 - pytorch_modeler.py - INFO - Epoch 207/300:train_loss:409.637090, tr_rec:398.653893, tr_eng:106.791734, tr_covd:60.804153, val_AUC:0.785926, val_pAUC:0.553386
100%|██████████| 20/20 [00:03<00:00,  5.53it/s]
2020-12-01 10:19:58,053 - pytorch_modeler.py - INFO - Epoch 208/300:train_loss:232.519029, tr_rec:399.608708, tr_eng:-1673.222640, tr_covd:46.515253, val_AUC:0.792741, val_pAUC:0.566870
100%|██████████| 20/20 [00:03<00:00,  5.56it/s]
2020-12-01 10:20:

100%|██████████| 20/20 [00:03<00:00,  5.46it/s]
2020-12-01 10:36:28,163 - pytorch_modeler.py - INFO - Epoch 234/300:train_loss:361.842267, tr_rec:394.273429, tr_eng:-328.565629, tr_covd:85.080605, val_AUC:0.653821, val_pAUC:0.637334
100%|██████████| 20/20 [00:03<00:00,  5.62it/s]
2020-12-01 10:37:07,525 - pytorch_modeler.py - INFO - Epoch 235/300:train_loss:363.109303, tr_rec:392.666172, tr_eng:-322.102455, tr_covd:530.675596, val_AUC:0.638859, val_pAUC:0.567423
100%|██████████| 20/20 [00:03<00:00,  5.33it/s]
2020-12-01 10:37:46,416 - pytorch_modeler.py - INFO - Epoch 236/300:train_loss:352.169858, tr_rec:394.264480, tr_eng:-425.887264, tr_covd:98.820258, val_AUC:0.696903, val_pAUC:0.636711
 81%|████████▏ | 66/81 [00:25<00:04,  3.48it/s]

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)